In [44]:
import pandas as pd
import json
import numpy as np
import os
import glob
import re
import urllib
from bs4 import BeautifulSoup
from pathlib import Path


In [45]:
def ig_unknown(file, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [46]:
def ig_isotope(measured_df, mat_list, hl_min, hl_max, name):
    precursores = mat_list.split(',')
    isotopes = []
    with open("database/irr_products.json") as f:
        irradiation_products = json.load(f)
    
    for precursor in precursores:
        try:
            isotopes += irradiation_products[precursor]
        
        except:
            irradiation_products[precursor] = load_irr_prod(precursor)
            with open("database/irr_products.json", 'w') as json_out:
                json.dump(irradiation_products, json_out)

    # get unique only
    isotopes = list(set(isotopes))

    db_parameters = ["half-life", "sigm_half-life", "lambda", "sigm_lambda", "isotope", "Ig", "sigm_Ig", "E_tab", "sigm_E_tab"]

    measured_df[[db_parameters]] = np.nan*np.ones(shape=(measured_df.shape[0], len(db_parameters)))

    for isotope in isotopes:
        measured_df = add_db_info(measured_df, isotope)
    
    measured_df.to_csv(f"out/{name}wDBinfo.csv")

In [47]:
def ig_nat_element(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [48]:
def ig_element_spec(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [49]:
def ig_material(file, mat_list, hl_min, hl_max):
    return "Feature not implemented yet. Write an issue to GitHub."

In [50]:
#def get_Igamma(file, material, hl_min, hl_max):
#    df = pd.read_csv(file,header=0,index_col=0)
#    name = file.split('.')[0]
#    mat_type, mat_list = material.split(',', maxindex=1)
#    switcher = {
#        0: ig_unknown(df, hl_min, hl_max, name),
#        "isotope": ig_isotope(df, mat_list, hl_min, hl_max, name), 
#        "element_natural": ig_nat_element(df, mat_list, hl_min, hl_max, name),
#        "element_spec": ig_spec_element(df, mat_list, hl_min, hl_max, name),
#        "material": ig_material(df, mat_list, hl_min, hl_max, name)
#    }
#    switcher.get(mat_type, "Invalid material given")

In [192]:
def append_Igamma_dir(parsed_dir):
    if not os.path.isdir(parsed_dir):
        raise Exception(f"Directory '{parsed_dir}' not found. Consider creating it and moving parsed files there.")

    Path("./reports_Ig").mkdir(parents=True, exist_ok=True)
    for parsed_file in glob.iglob(f"{parsed_dir}/*.csv"):
        append_Igamma(parsed_file)

In [193]:
def append_Igamma(file):
    print(file)
    A, element = re.split(r'(\d+)(\w+)', file)[-3:-1]
    print(A)
    print(element)

    Path("./ig_db").mkdir(parents=True, exist_ok=True)
    if os.path.isfile(f"ig_db/{A}{element}.csv"):
        ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
        print("Reading the csv with gammas.")
    else:
        Path("./downloads").mkdir(parents=True, exist_ok=True)
        if os.path.isfile(f"downloads/{A}{element}.html"):
            extract_Igamma(A, element)
            ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
            print("Reading the csv with gammas.")
        else:
            download(A, element)
            extract_Igamma(A, element)
            ig_df = pd.read_csv(f"ig_db/{A}{element}.csv", header=0, index_col=0)
            print("Reading the csv with gammas.")
    
    parsed_df = pd.read_csv(file, index_col=0)
    joined_df = add_Ig(parsed_df, ig_df)
    
    Path("./with_Ig").mkdir(parents=True, exist_ok=True)
    joined_df.to_csv(f"with_Ig/{A}{element}.csv")


In [194]:
def add_Ig(df, ig):
    e_col = []
    ig_col = []

    a = np.empty((df.shape[0], 2))
    a[:] = np.nan
    df[["E", "Ig"]] = a
    # UGLY, never for-loop in pandas
    for row_no in range(df.shape[0]):
        for ig_row_no in range(ig.shape[0]):
            #print((df["Energy"] - df["FWHM"]).iloc[row_no])
            #print(ig["E"].iloc[ig_row_no])
            #print((df["Energy"] - df["FWHM"]).iloc[row_no] > ig["E"].iloc[ig_row_no])
            if (df["Energy"] - df["FWHM"]).iloc[row_no] > ig["E"].iloc[ig_row_no]:
                continue
            elif (df["Energy"] + df["FWHM"]).iloc[row_no] < ig["E"].iloc[ig_row_no]:
                break
            else:
                df.loc[row_no, ["E", "Ig"]] = ig.loc[ig_row_no, ["E", "Ig"]]
                #print()
                #ig = ig.drop(ig.index[0:ig_row_no])
            
    return df

In [195]:
df = pd.read_csv("parsed_reports/239Pu.csv", index_col=0)


In [196]:
a = np.empty((df.shape[0], 2))
a[:] = np.nan
df[["E", "Ig"]] = a

In [197]:
#df["Energy"]+df["FWHM"]
#df[["Energy"]]
ig=pd.read_csv("ig_db/239Pu.csv", index_col=0)

In [198]:
ig

,E,sigm_E,Ig,sigm_Ig
0,0.0768,5,NaN,NaN
1,12.9630,2,NaN,NaN
2,30.0370,3,2.170000e-04,6.0
3,38.6610,2,1.050000e-02,2.0
4,40.4100,5,1.621000e-04,6.0
...,...,...,...,...
198,986.9100,4,2.100000e-08,4.0
199,992.6400,3,2.700000e-08,4.0
200,1005.8000,2,1.800000e-08,3.0
201,1009.4000,3,1.400000e-08,3.0


In [199]:
df.loc[2, ["E", "Ig"]] = ig.loc[3, ["E", "Ig"]]

In [223]:
wig = pd.read_csv("with_Ig/152Eu.csv", index_col=0)
wig[["Energy", "E", "Ig"]]

,Energy,E,Ig
0,25.31,NaN,NaN
1,40.29,NaN,NaN
2,75.20,NaN,NaN
3,84.81,NaN,NaN
4,271.33,271.13100,0.07292
5,275.60,275.44910,0.03352
6,296.09,295.93921,0.44700
7,315.52,316.30000,0.00211
8,324.90,324.83000,0.07200
9,329.56,329.42520,0.12800


In [218]:
def getZ(element):
    element_Z ={
        "Eu": 63,
        "U": 92,
        "Pu": 94
    }
    return element_Z[element]

In [219]:
def download(A, element):
    url = f"http://nucleardata.nuclear.lu.se/toi/nuclide.asp?iZA={getZ(element)}0{A}"
    urllib.request.urlretrieve (url, f"downloads/{A}{element}.html")
    print(f"File {A}{element}.html downloaded.")

In [220]:
def extract_Igamma(A, element):
    html_file = open(f"downloads/{A}{element}.html", "r")
    soup = BeautifulSoup(html_file.read(), 'lxml')
    
    gammas_table = soup.find_all("table")[4]
    gammas_rows = gammas_table.find_all('tr')[3:-1]
    energy = []
    sigm_energy = []
    i = []
    sigm_i = []

    for row in gammas_rows:
        cells = row.find_all('td')
        
        e_val = cells[0].get_text(strip=True)
        i_val = cells[1].get_text(strip=True)
        try:
            ig_val = float(i_val[:-1])
            sigm_ig_val = float(i_val[-1])
        except:
            ig_val = float('NaN')
            sigm_ig_val = float('NaN')
       
        energy.append(float(e_val[:-1]))
        sigm_energy.append(int(e_val[-1]))
        i.append(ig_val)
        
        sigm_i.append(sigm_ig_val)


    df_dict = {
        "E": energy,
        "sigm_E": sigm_energy, 
        "Ig": i,
        "sigm_Ig": sigm_i
        }
    df = pd.DataFrame(df_dict)
    df_name = f'ig_db/{A}{element}.csv'
    df.to_csv(df_name)
   
    print(f"Ig extracted from file 'downloads/{A}{element}.html' into '{df_name}'.")

In [221]:
html_doc = open("downloads/238U.html", "r")

soup = BeautifulSoup(html_doc.read(), 'lxml')
gammas_table = soup.find_all("table")[4]
gammas_rows = gammas_table.find_all('tr')[3:-1]
gamma_energy_list = []
gamma_i_list = []

for row in gammas_rows:
    cells = row.find_all('td')
    e_val = cells[0].get_text()
    print(re.split(r'\\', e_val))

    gamma_energy_list.append(e_val)
    gamma_i_list.append(cells[1].get_text(strip=False))


print(gamma_energy_list)
print(gamma_i_list)

FileNotFoundError: [Errno 2] No such file or directory: 'downloads/238U.html'

In [224]:
append_Igamma_dir("parsed_reports")

parsed_reports/152Eu.csv
152
Eu
Reading the csv with gammas.
